# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [3]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [4]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
table = soup.find_all('tr', class_='data-row')
table

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [17]:
# Create an empty list
mars_data = []

# Loop through the scraped data to create a list of rows
for row in table:
    row_data = [data.text for data in row.find_all('td')]
    mars_data.append(row_data)

In [18]:
mars_data

[['2', '2012-08-16', '10', '155', '6', '-75.0', '739.0'],
 ['13', '2012-08-17', '11', '156']]

In [8]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_df = pd.DataFrame(mars_data, columns=['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure'])

In [15]:
# Confirm DataFrame was created successfully
mars_df.head(10)

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,None,None,None


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [10]:
# Examine data type of each column
mars_df.dtypes

id                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object

In [14]:
# Change data types for data analysis
mars_df = mars_df.astype({'terrestrial_date': 'datetime64[ns]',
                          'sol': 'int64',
                          'ls': 'int64',
                          'month': 'int64',
                          'min_temp': 'float64',
                          'pressure': 'float64'})

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
mars_df['month'].groupby(mars_df['month']).count()

In [ ]:
# 2. How many Martian days' worth of data are there?
len(mars_df['terrestrial_date'].unique())

In [ ]:
# 3. What is the average low temperature by month?
avg_low_by_month = mars_df[['min_temp']].groupby(mars_df['month']).mean()
avg_low_by_month

In [ ]:
# Plot the average temperature by month
plt.bar(avg_low_by_month.index, avg_low_by_month['min_temp'])
plt.xticks(range(1,13))
plt.xlabel('Month')
plt.ylabel('Temperature (Celcius)')
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
avg_temp_sorted = avg_low_by_month.sort_values(['min_temp'])
avg_temp_sorted.plot.bar(legend=False)
plt.xlabel('Month')
plt.ylabel('Temperature (Celcius)')
plt.show()

In [ ]:
# 4. Average pressure by Martian month
avg_pressure_by_month = mars_df[['pressure']].groupby(mars_df['month']).mean()
avg_pressure_by_month

In [ ]:
# Plot the average pressure by month
avg_pressure_sorted = avg_pressure_by_month.sort_values(['pressure'])
avg_pressure_sorted.plot.bar(legend=False)
plt.xlabel('Month')
plt.ylabel('Atmospheric Pressure')
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
plt.plot(mars_df.index, mars_df['min_temp'])
plt.xlabel('Number of terrestrial days')
plt.ylabel('Minimum temerature (Celcius)')
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv('Resources/mars.csv', index = False)

In [ ]:
browser.quit()